In [1]:

from time import time

import numpy as np
#import tensorflow_addons as tfa
import pandas as pd
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow import keras
from tensorflow.python.keras.callbacks import TensorBoard
from tqdm.notebook import tqdm
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
#read train dataset
tf.random.set_seed(5)
print("open dataset")
train = pd.read_csv("../dataset/goodreads_train.csv")

open dataset


In [3]:
# Load a np_archive of review_text col of train dataset preprocess in main.py
train_prepro = pd.DataFrame(data=np.load(file="../vocabulaires/prepro_train_archive_NEG_lem.csv.npy", allow_pickle=True), columns=['review_text'])['review_text']


In [4]:
# add review text col to train dataset
train['review_text'] = train_prepro

In [6]:
rating = keras.utils.to_categorical(train['rating'], num_classes=6)
#create model layers
inputs = keras.Input(shape=(1,), dtype=tf.string) # text
inputs2 = keras.Input(shape=1, dtype=tf.float32) # n_comment
inputs3 = keras.Input(shape=1, dtype=tf.float32) # n_votes
#create vectorize layer, to transform words in integer
vectorize_layer = keras.layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    output_mode='int',
    output_sequence_length=1400,
    vocabulary=np.load('../vocabulaires/voc_lemm_without_NP.npy')
)(inputs)

embedding = keras.layers.Embedding(391583, 50, input_length=1400, batch_size=100)(vectorize_layer)

flatten = keras.layers.Flatten()(embedding)
#layer = keras.layers.Dense(200, activation=tf.keras.activations.softmax)(flatten)

conc = keras.layers.concatenate([flatten, inputs2,inputs3])
layer2 = keras.layers.Dense(300, activation=tf.keras.activations.softmax)(conc)
layer3 = keras.layers.Dense(300, activation=tf.keras.activations.softmax)(layer2)
outputs = keras.layers.Dense(6, activation=tf.keras.activations.relu)(layer3)

In [7]:
model = keras.Model(inputs=[inputs, inputs2, inputs3], outputs=outputs, name="mnist_model")
tensorboard = TensorBoard(log_dir="logs".format(time()))

In [8]:
#for learning_rate in learning_rates:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[
                  tf.keras.metrics.categorical_accuracy
                       ], run_eagerly=True
              )

In [9]:
class_weights = compute_class_weight(class_weight='balanced',classes= np.unique(train['rating']), y = train['rating'])
di = {}
for i in range(len(class_weights)):
    di[i] = class_weights[i]
model.save("../models_trained/pmc4")

In [17]:
model.fit([train['review_text'], train['n_comments'], train['n_votes']], rating, epochs=10,
                  callbacks=[
                      tf.keras.callbacks.TensorBoard(log_dir="logs"),
                  ],
                  batch_size=3000, class_weight=di, shuffle=True, validation_split=0.2
                  )

Epoch 1/10
240/240 [==============================] - 88s 366ms/step - loss: 0.6559 - categorical_accuracy: 0.4438 - val_loss: 1.2189 - val_categorical_accuracy: 0.3685
Epoch 2/10
240/240 [==============================] - 89s 369ms/step - loss: 0.6496 - categorical_accuracy: 0.4795 - val_loss: 1.2384 - val_categorical_accuracy: 0.3672
Epoch 3/10
240/240 [==============================] - 89s 369ms/step - loss: 0.6486 - categorical_accuracy: 0.4873 - val_loss: 1.2353 - val_categorical_accuracy: 0.3530
Epoch 4/10
240/240 [==============================] - 88s 368ms/step - loss: 0.6450 - categorical_accuracy: 0.4921 - val_loss: 1.2447 - val_categorical_accuracy: 0.3648
Epoch 5/10
240/240 [==============================] - 91s 381ms/step - loss: 0.6445 - categorical_accuracy: 0.5029 - val_loss: 1.2529 - val_categorical_accuracy: 0.3711
Epoch 6/10
240/240 [==============================] - 92s 384ms/step - loss: 0.6423 - categorical_accuracy: 0.5046 - val_loss: 1.2587 - val_categorical_acc

In [18]:
model.save("../models_trained/pmc4_train")

INFO:tensorflow:Assets written to: ../models_trained/model_with_embedding2\assets


In [19]:
#read test dataset
test = pd.read_csv("../dataset/goodreads_test.csv")

In [20]:
# Load a np_archive of review_text col of train dataset preprocess in main.py
test_prepro = pd.DataFrame(data=np.load(file="../vocabulaires/prepro_test_archive_PN_less.npy", allow_pickle=True), columns=['review_text'])['review_text']
test['review_text'] = test_prepro

In [ ]:
restest = model.predict([test['review_text'], test['n_comments'], test['n_votes']])

 5275/14939 [=========>....................] - ETA: 2:31

In [ ]:
# reverse keras.utils.to_categorical for kaggle submission
ff = []
for line in tqdm(restest):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
data = np.array(ff)

In [ ]:
test['rating'] = data

In [ ]:
# create a dataframe for kaggle
id = test['review_id'].to_numpy()
rating = test['rating'].to_numpy()
df = pd.DataFrame( columns=['review_id', 'rating'])

In [ ]:
df['review_id'] = id
df['rating'] = rating

In [ ]:
# create a csv for submission
df.to_csv('pmc4_model.csv',index=False )